In [1]:
!cp -r ../input/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./
%config Completer.use_jedi = False
%load_ext Cython
import pandas as pd
import numpy as np
import scipy.sparse as sps
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [2]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:746:
/opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In function ‘__

In [3]:
ICM_type_df = pd.read_csv("/kaggle/input/competition-data/data_ICM_type.csv")
ICM_type_df

items = ICM_type_df.item_id
features = ICM_type_df.feature_id
data = ICM_type_df.data
ICM_type = sps.csr_matrix((data, (items, features)))
ICM_type = ICM_type.astype(dtype = np.int32)

n_users = 41629
n_itemsFromICM = ICM_type.shape[0]
ICM_type.shape

(27968, 8)

In [4]:
URM_train = sps.load_npz("/kaggle/input/split8020/URM_Train80.npz")
URM_train = sps.csr_matrix((URM_train.data, URM_train.indices, URM_train.indptr), shape=(n_users, n_itemsFromICM))
URM_train

<41629x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 1243712 stored elements in Compressed Sparse Row format>

In [5]:
URM_valid = sps.load_npz("/kaggle/input/split8020/URM_Valid20.npz")
URM_valid = sps.csr_matrix((URM_valid.data, URM_valid.indices, URM_valid.indptr), shape=(n_users, n_itemsFromICM))
URM_valid

<41629x27968 sparse matrix of type '<class 'numpy.float64'>'
	with 310928 stored elements in Compressed Sparse Row format>

In [6]:
URM_train_stacked = sps.vstack([URM_train, ICM_type.T])
URM_train_stacked = sps.csr_matrix(URM_train_stacked)

In [7]:
#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 321 ( 0.8%) Users that have less than 1 test interactions


In [8]:
import implicit
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
import scipy.sparse as sps
class ImplicitALSRecommender(BaseMatrixFactorizationRecommender):
    """ImplicitALSRecommender recommender"""

    RECOMMENDER_NAME = "ImplicitALSRecommender"

    def fit(self,
            factors=100,
            regularization=0.01,
            use_native=True, use_cg=True, use_gpu=False,
            iterations=15,
            calculate_training_loss=False, num_threads=0,
            confidence_scaling=None,
            icm_coeff = 1,
            **confidence_args
            ):
        self.rec = implicit.als.AlternatingLeastSquares(factors=factors, regularization=regularization,
                                                        use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
                                                        iterations=iterations,
                                                        calculate_training_loss=calculate_training_loss,
                                                        num_threads=num_threads,
                                                        random_state=5)
        self.rec.fit(confidence_scaling(self.URM_train, **confidence_args), show_progress=self.verbose)

        self.USER_factors = self.rec.user_factors
        self.ITEM_factors = self.rec.item_factors

In [9]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [10]:
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping
from Recommenders.Recommender_utils import check_matrix

def linear_scaling_confidence(URM_train, alpha):
    C = check_matrix(URM_train.T, format="csr", dtype=np.float32)
    C.data = 1.0 + alpha * C.data

    return C

In [11]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

slim_bpr = SLIM_BPR_Cython(URM_train)
slim_bpr.fit(epochs=650, sgd_mode = "sgd", topK = 483, lambda_i = 0.0006712905081189398, lambda_j = 0.06584150350451998, learning_rate = 0.0036482363905043207)

SLIM_BPR_Recommender: URM Detected 3461 (12.4%) items with no interactions.
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 30777.00 MB (95.85%) of 32110.00 MB, required is 3128.84 MB. Using dense matrix.
Processed 41629 (100.0%) in 0.44 sec. BPR loss is 2.67E-03. Sample per second: 93917
SLIM_BPR_Recommender: Epoch 1 of 650. Elapsed time 0.26 sec
Processed 41629 (100.0%) in 0.70 sec. BPR loss is 1.48E-02. Sample per second: 59209
SLIM_BPR_Recommender: Epoch 2 of 650. Elapsed time 0.52 sec
Processed 41629 (100.0%) in 0.96 sec. BPR loss is 3.25E-02. Sample per second: 43491
SLIM_BPR_Recommender: Epoch 3 of 650. Elapsed time 0.78 sec
Processed 41629 (100.0%) in 1.22 sec. BPR loss is 5.63E-02. Sample per second: 34243
SLIM_BPR_Recommender: Epoch 4 of 650. Elapsed time 1.03 sec
Processed 41629 (100.0%) in 0.47 sec. BPR loss is 8.26E-02. Sample per second: 88784
SLIM_BPR_Recommender: Epoch 5 of 650. Elapsed time 1.29 sec
Processed 41629 (100.0%) in 0.72 sec

In [12]:
ImplicitALS = ImplicitALSRecommender(URM_train)
ImplicitALS.fit(iterations= int(111.5), factors= int(64.3), regularization= 0.0001,use_gpu=False, num_threads=4, confidence_scaling=linear_scaling_confidence,**{"alpha":5.988})

ImplicitALSRecommender: URM Detected 3461 (12.4%) items with no interactions.


  0%|          | 0/111 [00:00<?, ?it/s]

In [13]:
SLIMElasticNet = SLIMElasticNetRecommender(URM_train_stacked)
SLIMElasticNet.fit(l1_ratio= 0.02663352009457766, alpha= 0.0010777130010163724, positive_only= True, topK= 15368)

SLIMElasticNetRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SLIMElasticNetRecommender: Processed 2764 ( 9.9%) in 5.00 min. Items per second: 9.21
SLIMElasticNetRecommender: Processed 5757 (20.6%) in 10.00 min. Items per second: 9.59
SLIMElasticNetRecommender: Processed 8766 (31.3%) in 15.00 min. Items per second: 9.74
SLIMElasticNetRecommender: Processed 11793 (42.2%) in 20.00 min. Items per second: 9.82
SLIMElasticNetRecommender: Processed 14812 (53.0%) in 25.01 min. Items per second: 9.87
SLIMElasticNetRecommender: Processed 17871 (63.9%) in 30.01 min. Items per second: 9.93
SLIMElasticNetRecommender: Processed 20885 (74.7%) in 35.01 min. Items per second: 9.94
SLIMElasticNetRecommender: Processed 23774 (85.0%) in 40.01 min. Items per second: 9.90
SLIMElasticNetRecommender: Processed 27968 (100.0%) in 42.40 min. Items per second: 10.99


In [14]:
RP3beta = RP3betaRecommender(URM_train)
RP3beta.fit(alpha= 0.8048209317587374, beta= 0.28523965008406016, topK= 56, implicit= False)

RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 2242.97 column/sec. Elapsed time 12.47 sec


In [15]:
EASE_R = EASE_R_Recommender(URM_train_stacked)
#%%
EASE_R.fit(topK = 329, l2_norm = 143.45076856759013, normalize_matrix = False)

EASE_R_Recommender: URM Detected 3 ( 0.0%) users with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 15.83 min


In [16]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

itemKNNCF = ItemKNNCFRecommender(URM_train)
itemKNNCF.fit(topK = 157, shrink = 463, similarity = 'cosine', normalize = True, feature_weighting = 'TF-IDF')

ItemKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 27968 (100.0%), 3885.05 column/sec. Elapsed time 7.20 sec


In [17]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

P3Alpha = P3alphaRecommender(URM_train)
P3Alpha.fit(topK= 113, alpha= 1.2429301516636324, implicit= True, normalize_similarity= True)

P3alphaRecommender: URM Detected 3461 (12.4%) items with no interactions.
P3alphaRecommender: Similarity column 27968 (100.0%), 2178.65 column/sec. Elapsed time 12.84 sec


In [18]:
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

pure_svd = PureSVDRecommender(URM_train)
pure_svd.fit(num_factors=40)

PureSVDRecommender: URM Detected 3461 (12.4%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 2.16 sec


In [19]:
result_dict, _ = evaluator_valid.evaluateRecommender(slim_bpr)
print(f"Slim_bpr: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(itemKNNCF)
print(f"ItemKNN: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(ImplicitALS)
print(f"Ials: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(SLIMElasticNet)
print(f"Slim_en: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(RP3beta)
print(f"Rp3: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(EASE_R)
print(f"Ease: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(P3Alpha)
print(f"P3Alpha: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(pure_svd)
print(f"Pure_svd: {result_dict['MAP'][10]}")

EvaluatorHoldout: Processed 41308 (100.0%) in 50.34 sec. Users per second: 821
Slim_bpr: 0.025328946812254113
EvaluatorHoldout: Processed 41308 (100.0%) in 41.36 sec. Users per second: 999
ItemKNN: 0.03118264785175418
EvaluatorHoldout: Processed 41308 (100.0%) in 39.18 sec. Users per second: 1054
Ials: 0.026376644441062098
EvaluatorHoldout: Processed 41308 (100.0%) in 52.90 sec. Users per second: 781
Slim_en: 0.036479666492979256
EvaluatorHoldout: Processed 41308 (100.0%) in 34.26 sec. Users per second: 1206
Rp3: 0.033997700779738994
EvaluatorHoldout: Processed 41308 (100.0%) in 54.01 sec. Users per second: 765
Ease: 0.035505503757293576
EvaluatorHoldout: Processed 41308 (100.0%) in 34.70 sec. Users per second: 1190
P3Alpha: 0.031520566176503284
EvaluatorHoldout: Processed 41308 (100.0%) in 37.68 sec. Users per second: 1096
Pure_svd: 0.019242253508678572


In [20]:
from Recommenders.Recommender_utils import check_matrix, similarityMatrixTopK
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender



class ItemKNNSimilarityHybridRecommender(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNSimilarityHybridRecommender
    Hybrid of two similarities S = S1*alpha + S2*(1-alpha)
    """

    RECOMMENDER_NAME = "ItemKNNSimilarityHybridRecommender"


    def __init__(self, URM_train, Similarity_1, Similarity_2, verbose = True):
        super(ItemKNNSimilarityHybridRecommender, self).__init__(URM_train, verbose = verbose)

        # CSR is faster during evaluation
        self.Similarity_1 = check_matrix(Similarity_1.copy(), 'csr')
        self.Similarity_2 = check_matrix(Similarity_2.copy(), 'csr')


    def fit(self, topK=100, alpha = 0.5, beta = 0):

        self.topK = topK
        self.alpha = alpha
        self.beta = beta


        W_sparse = self.Similarity_1*self.alpha + self.Similarity_2*self.beta

        self.W_sparse = similarityMatrixTopK(W_sparse, k=self.topK)
        self.W_sparse = check_matrix(self.W_sparse, format='csr')

In [21]:
slim_matrix = SLIMElasticNet.W_sparse
rp3_matrix = RP3beta.W_sparse
bpr_matrix = slim_bpr.W_sparse
knn_matrix = itemKNNCF.W_sparse

In [22]:
slimEn_itemKnn = ItemKNNSimilarityHybridRecommender(URM_train, slim_matrix, knn_matrix)
slimEn_itemKnn.fit(topK= 2337, alpha= 52.66663460245183, beta= 0.7828876171060497)

ItemKNNSimilarityHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.


In [23]:
rp3_slimBPR = ItemKNNSimilarityHybridRecommender(URM_train, rp3_matrix, bpr_matrix)
rp3_slimBPR.fit(topK= 1686, alpha= 89.07099310062463, beta= 0.3204897377005579)

ItemKNNSimilarityHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.


In [24]:
result_dict, _ = evaluator_valid.evaluateRecommender(slimEn_itemKnn)
print(f"SlimEn_itemKnn: {result_dict['MAP'][10]}")

result_dict, _ = evaluator_valid.evaluateRecommender(rp3_slimBPR)
print(f"Rp3_slimBPR: {result_dict['MAP'][10]}")

EvaluatorHoldout: Processed 41308 (100.0%) in 55.09 sec. Users per second: 750
SlimEn_itemKnn: 0.0364991062110246
EvaluatorHoldout: Processed 41308 (100.0%) in 48.35 sec. Users per second: 854
Rp3_slimBPR: 0.034372192019377606


In [25]:
from Recommenders.Recommender_utils import check_matrix, similarityMatrixTopK
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender



class ItemKNNSimilarityHybridRecommender(BaseItemSimilarityMatrixRecommender):
    """ ItemKNNSimilarityHybridRecommender
    Hybrid of two similarities S = S1*alpha + S2*(1-alpha)
    """

    RECOMMENDER_NAME = "ItemKNNSimilarityHybridRecommender"


    def __init__(self, URM_train, Similarity_1, Similarity_2, verbose = True):
        super(ItemKNNSimilarityHybridRecommender, self).__init__(URM_train, verbose = verbose)

        # CSR is faster during evaluation
        self.Similarity_1 = check_matrix(Similarity_1.copy(), 'csr')
        self.Similarity_2 = check_matrix(Similarity_2.copy(), 'csr')


    def fit(self, topK=100, alpha = 0.5, beta = 0):

        self.topK = topK
        self.alpha = alpha
        self.beta = beta


        W_sparse = self.Similarity_1*self.alpha + self.Similarity_2*self.beta

        self.W_sparse = similarityMatrixTopK(W_sparse, k=self.topK)
        self.W_sparse = check_matrix(self.W_sparse, format='csr')

In [26]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3, recommender_4):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        self.recommender_4 = recommender_4

        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0, gamma = 0, theta = 0):

        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.theta = theta

        self.norm = norm
        


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)
        item_weights_4 = self.recommender_4._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        norm_item_weights_4 = LA.norm(item_weights_4, self.norm)
       
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
          
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * self.gamma + item_weights_4 / norm_item_weights_4 * self.theta

        return item_weights

In [27]:
slimEn_itemKnn = ItemKNNSimilarityHybridRecommender(URM_train, slim_matrix, knn_matrix)
slimEn_itemKnn.fit(topK= 2337, alpha= 52.66663460245183, beta= 0.7828876171060497)

ItemKNNSimilarityHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.


In [28]:
final_recommender = DifferentLossScoresHybridRecommender(URM_train, slimEn_itemKnn, rp3_slimBPR, EASE_R, ImplicitALS)
final_recommender.fit(norm= 1, alpha= 0.5971417607077779, beta= 0.2522239844950351, gamma= 0.0016350629574128922, theta= 0.1384291098602536)

DifferentLossScoresHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.


In [29]:
result_dict, _ = evaluator_valid.evaluateRecommender(final_recommender)
print(f"Final_recommender: {result_dict['MAP'][10]}")

EvaluatorHoldout: Processed 41308 (100.0%) in 1.93 min. Users per second: 357
Final_recommender: 0.036769072941785906
